In [1]:
# 데이터를 다뤄보자. 
# 파티셔닝은 DataFrame과 DataSet이 물리적으로 배치되는 형태를 정의한다.
# 파티셔닝 스키마는 파티션을 배치하는 방법을 정의한다. 일단 해 보자.

#DataFrame 만든다. 전에 썼던 항공데이터다.
df = spark.read.format("json").load("/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

In [2]:
# 스키마 찍어보자. 스키마는 DataFrame의 컬럼 이름이랑 데이터 타입을 정의한다. 지금은 데이터 가져온 거에서 얻었다!
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [3]:
# 이렇게도 해보자. 위에 거랑 좀 생긴 거만 좀 다르게 나온다. 

spark.read.format("json").load("/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json").schema

StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

In [4]:
# 스키마는 여러 개의 StructureField 타입으로 구성된 StructType 객체란다. 세 번째 값은 null 되냐 안되냐임.
# 스키마를 직접 만들어보자. 필요한 타입들을 갖고 와야된다.

from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField("DEST_COUNTRY_NAME", StringType(), True),
    StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
    StructField("count", LongType(), False, metadata={"hello":"world"})
])

# 내가 만든 스키마를 주고 데이터를 가져오자!
df = spark.read.format("json").schema(myManualSchema)\
.load("/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

In [5]:
# 잘 됐나 찍어보자!

df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [6]:
# 이번에는 칼럼에 대해 알아보자. 컬럼은 로우에서, 로우는 DataFrame에서 얻는다.
# col, column으로 컬럼 생성이나 참조한다. 

from pyspark.sql.functions import col, column

df.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [7]:
# 컬럼은 그냥 표현식일 뿐이란다. 뭔 소린가 아직 모르겠다. 이 둘이 같댄다.

(((col("someCol")+5)*200)-6) < col("otherCol")

Column<b'((((someCol + 5) * 200) - 6) < otherCol)'>

In [8]:
from pyspark.sql.functions import expr

expr("(((someCol+5)*200)-6)<otherCol")

Column<b'((((someCol + 5) * 200) - 6) < otherCol)'>

In [9]:
# 이번에는 Row를 보자. 이렇게하면 DataFrame의 첫 번째 Row를 보여준다.

df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [10]:
# Row는 스키마에 맞게 생성되어야 한다. 순서까지도!

from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

# 뭐 들었나 꺼내볼 때는 이렇게 한다. 위치를 지정한다! 타입은 올바른 걸로 자동 변환된단다.
myRow[0]

'Hello'

In [11]:
# 원시(?) 데이터에서 DataFrame을 생성할 수도 있다는데? 아 이게 그거구나.

df = spark.read.format("json").load("/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("dfTable")

In [13]:
# Row 객체를 가진 Seq 타입을 변환해서 DataFrame을 생성할 수도 있다고 함. 일단 시키는대로 해 봄.

# 쓸 것 이것저것 임포트하고
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, LongType

# 임포트한 걸로 직접 스키마 만들고
myManualSchema = StructType([
    StructField("some", StringType(), True),
    StructField("cols", StringType(), True),
    StructField("names", LongType(), False)
])

# Row 만들어서 내 스키마에다 집어넣어 DataFrame을 만든다. 띄워보자.
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-----+----+-----+
| some|cols|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



In [16]:
# 내가 만든 DataFrame 가지고 select 날려보자!

df.select("DEST_COUNTRY_NAME").show(10)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
|    United States|
|    United States|
|       Costa Rica|
|          Senegal|
|          Moldova|
+-----------------+
only showing top 10 rows



In [17]:
# 다른 것도 뽑아보자.

df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(10)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
|    United States|            Ireland|
|            Egypt|      United States|
|    United States|              India|
|    United States|          Singapore|
|    United States|            Grenada|
|       Costa Rica|      United States|
|          Senegal|      United States|
|          Moldova|      United States|
+-----------------+-------------------+
only showing top 10 rows



In [18]:
# 컬럼을 참조하는 법들

df.select(
    expr("DEST_COUNTRY_NAME"),
    col("DEST_COUNTRY_NAME"),
    column("DEST_COUNTRY_NAME")
).show(10)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
|    United States|    United States|    United States|
|            Egypt|            Egypt|            Egypt|
|    United States|    United States|    United States|
|    United States|    United States|    United States|
|    United States|    United States|    United States|
|       Costa Rica|       Costa Rica|       Costa Rica|
|          Senegal|          Senegal|          Senegal|
|          Moldova|          Moldova|          Moldova|
+-----------------+-----------------+-----------------+
only showing top 10 rows



In [21]:
# 컬럼을 참조하는 경우랑 그냥 문자열을 섞어 쓰면 에러난다! 둘 중 하나만 해라...
# 라고 책에는 말했는데 에러 안 나는데????

df.select(col("DEST_COUNTRY_NAME"), "DEST_COUNTRY_NAME")

DataFrame[DEST_COUNTRY_NAME: string, DEST_COUNTRY_NAME: string]

In [26]:
# 컬럼 이름 다르게 지정해서 출력할 수도 있다. SQL 쿼리 날릴 때처럼? 그런데 이 때는 expr 표현식을 써야하나보다. 그냥 문자열이나 col()로 불러오니 에러난다!

df.select(expr("DEST_COUNTRY_NAME AS destination")).show(10)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
|United States|
|        Egypt|
|United States|
|United States|
|United States|
|   Costa Rica|
|      Senegal|
|      Moldova|
+-------------+
only showing top 10 rows



In [27]:
# 다른 이름으로 기껏 불러와놓고 다시 원래대로 띄우기

df.select(expr("DEST_COUNTRY_NAME AS destination").alias("DEST_COUNTRY_NAME")).show(10)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
|    United States|
|    United States|
|       Costa Rica|
|          Senegal|
|          Moldova|
+-----------------+
only showing top 10 rows



In [29]:
# expr 자주 쓰기 때문에 아예 메서드가 따로 나왔나보다. 아까 문자열로 해봤다가 빠꾸먹은 것 받아준다? 더 간단한 듯.

df.selectExpr("DEST_COUNTRY_NAME as newColumnName").show(10)

+-------------+
|newColumnName|
+-------------+
|United States|
|United States|
|United States|
|        Egypt|
|United States|
|United States|
|United States|
|   Costa Rica|
|      Senegal|
|      Moldova|
+-------------+
only showing top 10 rows



In [30]:
# 새로운 컬럼을 추가해보자. 출발지랑 도착지가 같은지 비교하고 알려주는 컬럼을 판다.
# 원래 있는 모든 컬럼 추가하고 boolean 값으로 비교값도 띄운다.

df.selectExpr("*", "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry").show(10)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
|            Egypt|      United States|   15|        false|
|    United States|              India|   62|        false|
|    United States|          Singapore|    1|        false|
|    United States|            Grenada|   62|        false|
|       Costa Rica|      United States|  588|        false|
|          Senegal|      United States|   40|        false|
|          Moldova|      United States|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 10 rows



In [33]:
# 거의 다 해외여행이다! 이번에는 count의 평균, 데이터 안에 있는 도착 나라의 수를 중복 빼고 출력한다.

df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(10)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



In [37]:
# 명시적인 값을 스파크한테 줘야된다. 리터럴이라는게 내가 준 값을 스파크가 이해하게 해준단다.
# 원래 있는 값 다 띄우고 1만 나오는 One 컬럼을 추가해본다.

from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(10)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
|    United States|            Ireland|  344|  1|
|            Egypt|      United States|   15|  1|
|    United States|              India|   62|  1|
|    United States|          Singapore|    1|  1|
|    United States|            Grenada|   62|  1|
|       Costa Rica|      United States|  588|  1|
|          Senegal|      United States|   40|  1|
|          Moldova|      United States|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 10 rows



In [40]:
# 신기하니까 2랑 3도 해본다.

df.select(expr("*"), lit(1).alias("One"), lit(2).alias("Two"), lit(3).alias("Three")).show(10)

+-----------------+-------------------+-----+---+---+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|Two|Three|
+-----------------+-------------------+-----+---+---+-----+
|    United States|            Romania|   15|  1|  2|    3|
|    United States|            Croatia|    1|  1|  2|    3|
|    United States|            Ireland|  344|  1|  2|    3|
|            Egypt|      United States|   15|  1|  2|    3|
|    United States|              India|   62|  1|  2|    3|
|    United States|          Singapore|    1|  1|  2|    3|
|    United States|            Grenada|   62|  1|  2|    3|
|       Costa Rica|      United States|  588|  1|  2|    3|
|          Senegal|      United States|   40|  1|  2|    3|
|          Moldova|      United States|    1|  1|  2|    3|
+-----------------+-------------------+-----+---+---+-----+
only showing top 10 rows



In [42]:
# 아무튼 리터럴Literal은 상수 값을 의미한다. 숫자만 된다는 것이다?
# 하지만 파이썬에서는 a도 잘 들어간다. SQL 한정인가보다.

df.select(expr("*"), lit("a").alias("One")).show(10)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  a|
|    United States|            Croatia|    1|  a|
|    United States|            Ireland|  344|  a|
|            Egypt|      United States|   15|  a|
|    United States|              India|   62|  a|
|    United States|          Singapore|    1|  a|
|    United States|            Grenada|   62|  a|
|       Costa Rica|      United States|  588|  a|
|          Senegal|      United States|   40|  a|
|          Moldova|      United States|    1|  a|
+-----------------+-------------------+-----+---+
only showing top 10 rows



In [43]:
# 위에 방법은 그냥 다 뺑끼고 사실은 컬럼 추가하는 메소드가 따로 있다. 원래 있는거 다 보여주고 추가한 것도 보여주는 애란다.

df.withColumn("numberOne", lit(1)).show(10)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
|    United States|            Ireland|  344|        1|
|            Egypt|      United States|   15|        1|
|    United States|              India|   62|        1|
|    United States|          Singapore|    1|        1|
|    United States|            Grenada|   62|        1|
|       Costa Rica|      United States|  588|        1|
|          Senegal|      United States|   40|        1|
|          Moldova|      United States|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 10 rows



In [44]:
# 아까 했던 도착지=출발지 비교도 이런식으로 추가해보자.
# 표현식을 써서 값을 줬다!

df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(10)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
|            Egypt|      United States|   15|        false|
|    United States|              India|   62|        false|
|    United States|          Singapore|    1|        false|
|    United States|            Grenada|   62|        false|
|       Costa Rica|      United States|  588|        false|
|          Senegal|      United States|   40|        false|
|          Moldova|      United States|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 10 rows



In [45]:
# 그런데 withColumn으로 이름 바꾸기도 된다. 대충 손에 익는 거 외워서 쓰면 되겠다.

df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

In [48]:
# 컬럼 이름 제대로 바꾸려면 사실 메서드가 따로 있다... 뭐야 이게
# 자세히 보니까 withColumn으로 바꾼 애는 뒤에 추가된 걸로 뜨는데, 이건 원래 컬럼 이름을 바꿔서 보여준다!

df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

In [57]:
# 공백이나 하이픈은 컬럼 이름에 못 쓴다. 이런 글자를 쓰려면 백틱을 써야한단다. 그런데 그거 맥에서 입력되나? 대소문자 써서 할 수 있네?
# 예를 들면 이렇게

dfWithLongColName = df.withColumn(
    "This Long Column-Name",
    expr("ORIGIN_COUNTRY_NAME"))

dfWithLongColName.selectExpr(
    "`This Long Column-Name`",
    "`This Long Column-Name` as `new Col`"
).show(10)

+---------------------+-------------+
|This Long Column-Name|      new Col|
+---------------------+-------------+
|              Romania|      Romania|
|              Croatia|      Croatia|
|              Ireland|      Ireland|
|        United States|United States|
|                India|        India|
|            Singapore|    Singapore|
|              Grenada|      Grenada|
|        United States|United States|
|        United States|United States|
|        United States|United States|
+---------------------+-------------+
only showing top 10 rows



In [58]:
# 스파크는 대소문자를 안 가린다. 이 뭐... 
# 그런데 따로 설정하면 대소문자 구분을 시킬 수 있단다. ㅇㅎ.

# -- SQL
# set spark.sql.caseSensitive true

In [59]:
# 컬럼 추가가 되면 제거도 돼야겠지

dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").columns

['count', 'This Long Column-Name']

In [63]:
# 데이터 타입을 형변환하는 건 cast()가 한다.
# 원래 count는 bigint인데 문자열로 바꾼 count2를 만들어보자!

df.withColumn("count2", col("count").cast("string"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, count2: string]

In [65]:
# 드디어 나왔다 filter!
# 정해진 조건으로 필터링을 한다. where랑 비슷하다.

df.filter(col("count")<2).show(10)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Croatia|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|Saint Vincent and...|      United States|    1|
|            Suriname|      United States|    1|
|       United States|             Cyprus|    1|
|        Burkina Faso|      United States|    1|
|            Djibouti|      United States|    1|
+--------------------+-------------------+-----+
only showing top 10 rows



In [66]:
# where로도 된다

df.where("count <2").show(10)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Croatia|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|Saint Vincent and...|      United States|    1|
|            Suriname|      United States|    1|
|       United States|             Cyprus|    1|
|        Burkina Faso|      United States|    1|
|            Djibouti|      United States|    1|
+--------------------+-------------------+-----+
only showing top 10 rows



In [72]:
# 필터를 여러개 걸 때는 이렇게 한다. 그런데 항상 유용하지는 않단다. 스파크가 필터 순서 상관 없이 다 필터링 작업하기 때문이란다. 
# 출발지가 크로아티아 아닌 곳만 띄워보자. 크무룩...

df.where(col("count") < 2).where(col("ORIGIN_COUNTRY_NAME") != "Croatia").show(10)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|Saint Vincent and...|      United States|    1|
|            Suriname|      United States|    1|
|       United States|             Cyprus|    1|
|        Burkina Faso|      United States|    1|
|            Djibouti|      United States|    1|
|       United States|            Estonia|    1|
+--------------------+-------------------+-----+
only showing top 10 rows



In [75]:
# 이번 장 왜케 기냐
# 고유값을 얻어보자. 파워 중복 제거! 사실 아까 해봤다... distinct()
# 출발, 도착 국가 수를 중복 빼고 센다.
# distinct()는 중복을 제거한 새로운 DataFrame을 반환하는 식으로 일한댄다.

df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

256

In [76]:
# 출발 국가만 세면

df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

In [77]:
# 무작위 샘플을 만들자. 랜덤잼 ㅋ
# sample() 이라는 메서드로 한다고 한다. ML용인가
# 추출 비율이나 복원/비복원 여부를 정할 수 있다는데 몰라 모른다고

# 이게 그 설정 값이란다
seed = 5
withReplacement = False
fraction = 0.5

# 저 설정대로 샘플 뽑으니 몇개냐면 
df.sample(withReplacement, fraction, seed).count()

126

In [78]:
# 임의 분할도 된다. 분할은 split인데
# 머신러닝에서 학습셋 검증셋 테스트셋 만들 때 주로 쓴단다

# 아까 맹근 seed 재활용 ㅋ
dataFrames = df.randomSplit([0.25, 0.75], seed)

# 나눈 결과는 배열로 오는구나... 당연히 0.25인 애가 작으니까 False 뜬다
dataFrames[0].count() > dataFrames[1].count()

False

In [88]:
# DataFrame은 맘대로 못 바꾼다.
# 뭔가 추가하고 싶으면 새로운 DataFrame을 만들어야한다. 원래 있는거랑 추가할 거를 통합한다든지.
# 당연히 통합될 놈들은 스키마랑 컬럼 수가 같아야한단다.

from pyspark.sql import Row

# 지정하기 귀찮았나보다 있는거 갖다 쓰네
schema = df.schema

# 통합할 새 로우들을 가라로 맹근다
newRows = [
    Row("New country1", "Other Country1", 5),
    Row("New country2", "Other Country2", 1),
    Row("New country3", "Other Country3", 2)
]

# 이건 뭐하는 거였더라
parallelizedRows = spark.sparkContext.parallelize(newRows)

# 새로운 DataFrame 만든다
newDF = spark.createDataFrame(parallelizedRows, schema)

# 합치자 와구와구 원래 나라 이름이 미국 아닌 애들만 뽑아다가 내가 새로 만든 Row들까지 합쳐놨다 하하
df.union(newDF)\
    .where("count = 1")\
    .where(col("ORIGIN_COUNTRY_NAME") != "United States")\
    .show()

# 저건 합친 결과를 그냥 띄웠을 뿐이다. 이거 두고두고 쓰고 싶으면 뷰로 등록해놓자.
newDF.createOrReplaceTempView("newDF")

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|     New country2|     Other Country2|    1|
+-----------------+-------------------+-----+



In [89]:
# 스파크가 정렬도 해준다 우왕 굿

df.sort("count").show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows



In [90]:
# 근데 방금 건 오름차순이라 뭔가 의미 없는 짓이야... 정렬 조건을 정하자
# 일단 컬럼 여러개로 정렬해보자

df.orderBy("count", "DEST_COUNTRY_NAME").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
|             Iraq|      United States|    1|
|           Kosovo|      United States|    1|
|            Malta|      United States|    1|
|          Moldova|      United States|    1|
|    New Caledonia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



In [92]:
# 에라이 미국... 정렬 기준을 내림차순으로 바꾸자

df.orderBy(col("count").desc()).show(10)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
|   United Kingdom|      United States|  2025|
|    United States|     United Kingdom|  1970|
|            Japan|      United States|  1548|
|    United States|              Japan|  1496|
|          Germany|      United States|  1468|
+-----------------+-------------------+------+
only showing top 10 rows



In [96]:
# 올 ㅋ 다른 것도 해 본다

df.orderBy(col("DEST_COUNTRY_NAME").asc()).show(10)

+-------------------+-------------------+-----+
|  DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-------------------+-------------------+-----+
|            Algeria|      United States|    4|
|             Angola|      United States|   15|
|           Anguilla|      United States|   41|
|Antigua and Barbuda|      United States|  126|
|          Argentina|      United States|  180|
|              Aruba|      United States|  346|
|          Australia|      United States|  329|
|            Austria|      United States|   62|
|         Azerbaijan|      United States|   21|
|            Bahrain|      United States|   19|
+-------------------+-------------------+-----+
only showing top 10 rows



In [109]:
# 값이 null인 애들을 잡아올 수도 있다...는데 에러난다 꺄르르
# 꼭 알아야 할 것 같은데 나중에 생각하자

In [113]:
# 파티션별로도 정렬을 시킬 수 있단다
# 불러올 때 말이지...

spark.read.format("json").load("/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/json/*-summary.json").sortWithinPartitions("count")

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [114]:
# 이거슨 튜닝과 최적화 때 다시 배우자
# 고지가 얼마 남지 않았다 헉헉
# Row 수를 제한해보자. 사실 show()할 때 숫자 주는거랑 같아보이긴 하는데

df.limit(5).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [115]:
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [121]:
# 둘이 수행 절차가 다른가 찍어보자

df.limit(5).explain()

== Physical Plan ==
CollectLimit 5
+- *(1) FileScan json [DEST_COUNTRY_NAME#40,ORIGIN_COUNTRY_NAME#41,count#42L] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>


In [123]:
df.explain()

== Physical Plan ==
*(1) FileScan json [DEST_COUNTRY_NAME#40,ORIGIN_COUNTRY_NAME#41,count#42L] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/Users/talysa/Documents/Study/Spark-The-Definitive-Guide/data/flight-data/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>


In [124]:
# limit를 걸면 뭔가 좀 다르시다? 는 일을 한 번 더 함 -_- 으로 밝혀짐
# 파티션 다시하기
# 왜때문에 다시 하냐면 자주 필터링하는 컬럼 기준으로 데이터 분할할라고
# 물리적 구성 제어의 영역 어쩌고...

# 파티션 수 가져온다
df.rdd.getNumPartitions()

1

In [125]:
# 파티션 5개로 맹근다
df.repartition(5)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [128]:
# 파티션 수 다시 확인한다?
df.rdd.getNumPartitions()

1

In [129]:
# 아 왜 또 1
# 액션을 안 시켜서 그런가보다
# 컬럼 기준으로 하면 어찌되냐

df.repartition(col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [130]:
# 파티션 수 다시 확인한다?
df.rdd.getNumPartitions()

1

In [131]:
# 여전히 1이시다
# 아무튼 파티션 수랑 주로 쓰는 컬럼 같이 지정할 수도 있단다

df.repartition(5, col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [132]:
# coalesce()는 데이터 셔플 안하고 파티션 합칠 때 쓴단다
# 굳이 5개로 나눴다가 2개로 합쳐봄

df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [135]:
# 아이고 의미 없다 
# 드라이버로 데이터 수집하는게 뭔지 모르는데 아무튼 collect()가 그거랜다

# 지금까지 쓰던 df에서 Row 10마리만 잡아다가 DataFrame 만듬
collectDF = df.limit(10)

In [139]:
# 5마리 잡아온 리스트를 만드름
collectDF.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [137]:
# show는 보여주기만 함
collectDF.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [143]:
# show()에 옵션 줘본다 근데 False는 무슨 역할인겨
collectDF.show(5, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [145]:
# Row 다 잡아온다 어헣 길어
df.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Sint Maarten', count=325),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Marshall Islands', count=39),
 

In [146]:
# 반복자 이터레이터 Iterator로 파티션 데이터를 드라이버에 갖다 줄라면
collectDF.toLocalIterator()

In [151]:
# 데이터 수집하는 거는 매우 큰 비용이 발생한댄다
# 드라이버가 뻗을 수도 있다고 함
# 연산이 병렬이 아니라서 매우 큰 처리 비용이 발생한단다 흠
# 그렇다고 해도 이거는 안 되고
# collectDF.collect(5)
# 이거는 된다
collectDF.take(5)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]